In [33]:
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered


In [34]:
import statsmodels.api as sm

In [35]:
import pandas as pd
import nilearn
import numpy as np
import glob 
import os
import pickle

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import pairwise_distances
from sklearn.utils import check_random_state
from sklearn.manifold import TSNE

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

from sklearn.svm import LinearSVC

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


from sklearn.model_selection import cross_val_score
from pathlib import Path

from scipy.stats import rankdata, ttest_rel, ttest_1samp

from matplotlib import pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms

import seaborn as sns

import nibabel as nib
from nilearn.input_data import NiftiLabelsMasker
from nilearn.plotting import plot_glass_brain, plot_stat_map, view_img, view_img_on_surf

from nltools.data import Brain_Data, Adjacency
from nltools.mask import roi_to_brain, expand_mask
from nltools.stats import fdr, threshold



%matplotlib inline


In [40]:
basepath = '/Users/gracer/Library/CloudStorage/OneDrive-UniversityofWyoming/0. Lab/M2AENAD Lab - Documents/RESEARCH/ABCD/Yana_SSIB_2024/'

In [41]:
df = pd.read_csv(os.path.join(basepath,'data','matched111724.csv'))  

In [42]:
# Get number of subjects
n = df.shape[0]

In [43]:
n

3700

In [44]:
list(df.columns)

['collection_id',
 'abcd_mx01_id',
 'dataset_id',
 'subjectkey',
 'src_subject_id',
 'demo_comb_income_v2',
 'race',
 'sex',
 'bkfs_soda',
 'interview_age',
 'anthroheightcalc',
 'rsfmri_c_ngd_ad_ngd_ad',
 'rsfmri_c_ngd_ad_ngd_cgc',
 'rsfmri_c_ngd_ad_ngd_ca',
 'rsfmri_c_ngd_ad_ngd_dt',
 'rsfmri_c_ngd_ad_ngd_dla',
 'rsfmri_c_ngd_ad_ngd_fo',
 'rsfmri_c_ngd_ad_ngd_n',
 'rsfmri_c_ngd_ad_ngd_rspltp',
 'rsfmri_c_ngd_ad_ngd_smh',
 'rsfmri_c_ngd_ad_ngd_smm',
 'rsfmri_c_ngd_ad_ngd_sa',
 'rsfmri_c_ngd_ad_ngd_vta',
 'rsfmri_c_ngd_ad_ngd_vs',
 'rsfmri_c_ngd_cgc_ngd_ad',
 'rsfmri_c_ngd_cgc_ngd_cgc',
 'rsfmri_c_ngd_cgc_ngd_ca',
 'rsfmri_c_ngd_cgc_ngd_dt',
 'rsfmri_c_ngd_cgc_ngd_dla',
 'rsfmri_c_ngd_cgc_ngd_fo',
 'rsfmri_c_ngd_cgc_ngd_n',
 'rsfmri_c_ngd_cgc_ngd_rspltp',
 'rsfmri_c_ngd_cgc_ngd_smh',
 'rsfmri_c_ngd_cgc_ngd_smm',
 'rsfmri_c_ngd_cgc_ngd_sa',
 'rsfmri_c_ngd_cgc_ngd_vta',
 'rsfmri_c_ngd_cgc_ngd_vs',
 'rsfmri_c_ngd_ca_ngd_ad',
 'rsfmri_c_ngd_ca_ngd_cgc',
 'rsfmri_c_ngd_ca_ngd_ca',
 'rsfmri_

In [45]:
nonBrain = [list(df.columns)[5], list(df.columns)[7] , list(df.columns)[9],list(df.columns)[-4]]

In [46]:
nonBrain 

['demo_comb_income_v2', 'sex', 'interview_age', 'bmi_perc']

In [47]:
ROIs = list(df.columns)[11:-8]

In [49]:
ROIS = nonBrain+ROIs

In [50]:
df['sex'] = df['sex'].replace({'M':0,'F':1})

In [51]:
df['ssb_group'] = df['ssb_group'].replace({'low':0,'high':1})

In [52]:
X = df[ROIS] #features

In [53]:
X.shape

(3700, 420)

In [54]:
# Transpose the DataFrame to make columns become rows
X_T = X.T

duplicates = X_T.duplicated(keep='first')


In [55]:
# Identifying columns to drop (all duplicates except the first occurrence)
cols_to_drop = X_T[duplicates].index


In [56]:
# Drop the duplicate columns from the original DataFrame
X_cleaned = X.drop(cols_to_drop, axis=1)

print(X_cleaned)

      demo_comb_income_v2  sex  interview_age  bmi_perc  \
0                       9    0            121  0.528241   
1                       9    1            115  0.436110   
2                       1    0            121  0.485172   
3                       8    1            118  0.784327   
4                       4    1            118  0.735084   
...                   ...  ...            ...       ...   
3695                    8    0            125  0.512591   
3696                    9    1            130  0.509432   
3697                    7    1            114  0.811480   
3698                   10    0            131  0.611673   
3699                    9    0            124  0.916606   

      rsfmri_c_ngd_ad_ngd_ad  rsfmri_c_ngd_ad_ngd_cgc  rsfmri_c_ngd_ad_ngd_ca  \
0                   0.374677                 0.156529               -0.021578   
1                   0.316619                 0.180278               -0.048049   
2                   0.285115                 0.1

In [58]:
X.shape

(3700, 420)

In [59]:
X_cleaned.shape

(3700, 342)

# Factor of interest
This is sometimes called the target. Right now I am using sex, but eventually we will change this to SSB group

In [60]:
y = df['ssb_group'] #target

In [61]:
y.value_counts()

1    1850
0    1850
Name: ssb_group, dtype: int64

# Train and test datasets
Get randomly generated train and test datasets
- Train 1 = train the model and feature elimination
- Train 2 = cross validate the model
- Test = test statistical differences

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y, random_state=42)


In [63]:
X_train1, X_train2, y_train1, y_train2 = train_test_split(X_train, y_train, random_state=42)


In [64]:
X_test.shape

(925, 342)

In [65]:
y_test.value_counts()

0    481
1    444
Name: ssb_group, dtype: int64

In [66]:
X_train.shape

(2775, 342)

In [67]:
X_train1.shape

(2081, 342)

In [68]:
y_train2.shape

(694,)

In [69]:
anova_filter = SelectKBest(f_classif)
svm = SVC(kernel='linear')

anova_svm = Pipeline([
    ('anova', anova_filter),
    ('svc', svm)
])
# Define a range of parameters for feature selection and SVM
param_grid = {
    'anova__k': [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60],  # Trying different numbers of top features
    'svc__C': [0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15],  # SVM regularization parameter
}

# Setup GridSearchCV
grid_search = GridSearchCV(anova_svm, param_grid=param_grid, cv=10, n_jobs=4)
grid_search.fit(X_train1, y_train1)


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('anova', SelectKBest()),
                                       ('svc', SVC(kernel='linear'))]),
             n_jobs=4,
             param_grid={'anova__k': [50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
                                      60],
                         'svc__C': [0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11,
                                    0.12, 0.13, 0.14, 0.15]})

In [ ]:
range(49)

In [70]:
bestK = []
bestC = []
for i in range(49):
    print(i)
    # Setup GridSearchCV
    grid_search = GridSearchCV(anova_svm, param_grid=param_grid, cv=10, n_jobs=4)
    grid_search.fit(X_train1, y_train1)
    bestK.append(grid_search.best_params_['anova__k'])
    bestC.append(grid_search.best_params_['svc__C'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


## Initial
* anova_initial = [10,20,30,40,50,60]
* svc_C_initial = [0.1, 1, 10, 100]
* Then narrowed to anova 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60
* svc c 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15
## First try
* Best parameters: {'anova__k': 51, 'svc__C': 0.15}
* Best cross-validation score: 0.5530157342657344## Second try
## 48 iterations 
* Best parameters: {'anova__k': 50, 'svc__C': 0.05}
* Best cross-validation score: 0.7112026131762974

In [71]:
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

Best parameters: {'anova__k': 50, 'svc__C': 0.05}
Best cross-validation score: 0.7112026131762974


In [ ]:
# Assuming anova_svm is your original pipeline
best_pipeline = Pipeline([
    ('anova', SelectKBest(f_classif)),  # You don't need to specify k here; it will be set by best_params_
    ('svc', SVC(kernel='linear'))      # No need to specify C here for the same reason
])

# Set the best parameters found for the entire pipeline
best_pipeline.set_params(**grid_search.best_params_)

# Now, retrain on the entire training set with the best parameters
best_pipeline.fit(X_train1, y_train1)

In [ ]:
from sklearn.metrics import accuracy_score

# Predict on the test set
y_pred = best_pipeline.predict(X_train2)

# Evaluate the model
test_accuracy = accuracy_score(y_train2, y_pred)
print("Test set accuracy:", test_accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_train2, y_pred, normalize = 'true')
print(cm)

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True,  cmap="Blues")
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
# Access the best set of parameters found by GridSearchCV
best_parameters = grid_search.best_params_
print("Best parameters found by GridSearchCV:", best_parameters)

# Access the best estimator directly
best_model = grid_search.best_estimator_
print("Best model:", best_model)


In [ ]:
# Access the feature selection step ('anova' in your case)
feature_selection_step = best_model.named_steps['anova']


In [ ]:
# Get the mask of selected features (boolean array)
selected_features_mask = feature_selection_step.get_support()



In [ ]:
selected_features_mask.shape

In [ ]:
selected_columns = X_train.columns[selected_features_mask]


In [ ]:
selected_columns

# Regression

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
X_regression = X_test[selected_columns]

In [ ]:
X_regression.shape

In [ ]:
X_trainReg, X_testReg, y_trainReg, y_testReg = train_test_split(X_regression, y_test, random_state=42)


In [ ]:
X_testReg.shape

In [ ]:
# Initialize the Logistic Regression model
model = LogisticRegression(max_iter=1000)

In [ ]:
# Fit the model on the training data
model.fit(X_trainReg, y_trainReg)

In [ ]:
# Predict on the test set
y_predReg = model.predict(X_testReg)

# Calculate the accuracy
accuracyReg = accuracy_score(y_testReg, y_predReg)
print(f"Accuracy: {accuracyReg}")


In [ ]:
# Coefficients
coefficients = model.coef_
# Intercepts
intercepts = model.intercept_

In [ ]:
coeff_df = pd.DataFrame(coefficients, columns=selected_columns)


In [ ]:
coeff_df['Intercept'] = intercepts

In [ ]:
class_names = ['low','high']  # Adjust as per your classes
coeff_df.index = class_names

print(coeff_df)


In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(coeff_df.iloc[:, :-1], annot=False, cmap='coolwarm')  # Exclude intercepts for visualization
plt.title('Coefficients of Multinomial Logistic Regression')
plt.ylabel('Class')
plt.xlabel('Feature')
plt.show()

In [ ]:
# Assuming X_trainReg and y_trainReg are your training data and labels
# Add constant to the features for the intercept
X_test_reg_sm = sm.add_constant(X_testReg)


In [ ]:
X_test_reg_sm.shape

In [ ]:
y_testReg

In [ ]:
y_testReg = y_testReg.replace({2:1})

In [ ]:
# Fit the multinomial logistic regression model
logit_model = sm.Logit(y_testReg, X_test_reg_sm)
result = logit_model.fit()
#model_sm = sm.MNLogit(y_testReg, X_test_reg_sm)
#result_sm = model_sm.fit(method='newton', maxiter=5000)


In [ ]:
# Summary of the model
# print(result_sm.summary())
print(result.summary())

rsfmri_cor_ngd_cerc_scs_vtdcrh


In [ ]:
# Making predictions
predictions = result.predict(X_test_reg_sm)

# Converting probabilities to class labels
class_predictions = np.where(predictions > 0.5, 1, 0)

In [ ]:
class_predictions

In [ ]:
array = y_testReg.to_numpy()

In [ ]:
dfPrevReal = pd.DataFrame({'real': array, 'pred': class_predictions})

In [ ]:
dfPrevReal

In [ ]:
from sklearn.metrics import jaccard_score

jaccard = jaccard_score(dfPrevReal['real'], dfPrevReal['pred'])
print("Jaccard Similarity Score:", jaccard)


In [ ]:
# Extract p-values for all coefficients in the model
p_values = result.pvalues

In [ ]:
from statsmodels.stats.multitest import multipletests

# Apply Benjamini-Hochberg FDR correction
reject, pvals_corrected, _, _ = multipletests(p_values.values.flatten(), alpha=0.05, method='fdr_bh')

# Reshape the corrected p-values to match the original shape
pvals_corrected_reshaped = pvals_corrected.reshape(p_values.shape)

# Create a DataFrame of the corrected p-values for easier interpretation
# corrected_pvalues_df = pd.DataFrame(pvals_corrected_reshaped, index=p_values.index, columns=p_values.columns)

In [ ]:
pvals_corrected_reshaped

# Notes
Have too many features per sample/group. Will need to do dimensionaity reduction to get ~ 50 features per 500 subjects

## Reminder
matrix = sub,row,col